<h2> Taller 8 - Pick and Place</h2>

<br>Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter.
<br> Desde CoppeliaSim, abrir la escena MTB_Pick_N_Place.ttt


In [256]:
# importamos las librerías necesarias
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico
import time

### 0. Verificamos que todo esté funcionando... 

El robot cuenta con una vensosa en el extremo. Ésta se activa automáticamente al acercarse al cubo (1 mm o menos) y se desactiva cuando el cubo se coloca sobre la plataforma (5 mm o menos).

In [257]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [258]:
def setEffector(val):
# acciona el efector final
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
    return res

In [259]:
# Requerimos los manejadores para las articulaciones y el Dummy
clientID = connect(19999)

retCode,effector=sim.simxGetObjectHandle(clientID,'effector',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'MTB_joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'MTB_joint3',sim.simx_opmode_blocking)
retCode,joint4=sim.simxGetObjectHandle(clientID,'MTB_joint4',sim.simx_opmode_blocking)
retCode,caja1=sim.simxGetObjectHandle(clientID,'Caja1',sim.simx_opmode_blocking)
retCode,caja2=sim.simxGetObjectHandle(clientID,'Caja2',sim.simx_opmode_blocking)
retCode,caja3=sim.simxGetObjectHandle(clientID,'Caja3',sim.simx_opmode_blocking)

print(effector, joint1, joint2, joint3, joint4, caja1, caja2, caja3)

conectado a 19999
38 21 22 23 34 35 36 37


In [260]:
# Enviamos las posiciones a las articulaciones
q = [0, 0, 0.0]

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


In [261]:
# Activamos o desactivamos el actuador final
setEffector(0)

0

###  1. Preparamos la información necesaria
A partir de la matriz de cinemática directa calculamos la cinemática inversa.

In [262]:
# 1. Las expresiones de cinemática directa

# En una actividad anterior se obtuvo la matriz de cinemática directa
# para el actuador final, como función de las posiciones de las articulaciones.

q1 = sp.symbols('q1') # ángulo de la articulación rotacional joint1, en radianes
q2 = sp.symbols('q2') # ángulo de la articulación rotacional joint2, en radianes
q3 = sp.symbols('q3') # posición de la articulación prismática joint3, en metros
q4 = sp.symbols('q4') # ángulo de la articulación rotacional joint4, en radianes

T = sp.Matrix([[sp.cos(q1 + q2 + q4), -sp.sin(q1 + q2 + q4), 0, 0.467*sp.cos(q1) + 0.4005*sp.cos(q1 + q2)],
            [sp.sin(q1 + q2 + q4), sp.cos(q1 + q2 + q4), 0, 0.467*sp.sin(q1) + 0.4005*sp.sin(q1 + q2)], 
            [0, 0, 1, 0.234 - q3], 
            [0, 0, 0, 1]])
T

Matrix([
[cos(q1 + q2 + q4), -sin(q1 + q2 + q4), 0, 0.467*cos(q1) + 0.4005*cos(q1 + q2)],
[sin(q1 + q2 + q4),  cos(q1 + q2 + q4), 0, 0.467*sin(q1) + 0.4005*sin(q1 + q2)],
[                0,                  0, 1,                          0.234 - q3],
[                0,                  0, 0,                                   1]])

###  2. Obtenemos la posición y orientación del punto de destino


In [263]:
# Definimos una función para construir la matriz de rotación
# a partir de los ángulos de euler

def matrixFromEuler(alpha, beta, gamma):
    # theta y alpha en radianes
    # d y a en metros
    Ra = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    Rb = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta), 0],
                   [0, 1, 0, 0],
                   [-sp.sin(beta), 0, sp.cos(beta), 0],
                   [0, 0, 0, 1]])
    Rc = sp.Matrix([[sp.cos(gamma), -sp.sin(gamma), 0, 0],
                   [sp.sin(gamma), sp.cos(gamma), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    T = Ra*Rb*Rc
    return T

In [264]:
matrixFromEuler(45*sp.pi/180, 0, 45*sp.pi/180)

Matrix([
[sqrt(2)/2, -sqrt(2)/2,          0, 0],
[      1/2,        1/2, -sqrt(2)/2, 0],
[      1/2,        1/2,  sqrt(2)/2, 0],
[        0,          0,          0, 1]])

In [265]:
# A partir de las coordenadas de posición y los ángulos de Euler
# es posible obtener la matriz de transformación

x = 0.5
y = 0.5
z = 0.1
alpha = 0
beta = 0
gamma = 45*sp.pi/180

t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)
D

Matrix([
[sqrt(2)/2, -sqrt(2)/2, 0, 0.5],
[sqrt(2)/2,  sqrt(2)/2, 0, 0.5],
[        0,          0, 1, 0.1],
[        0,          0, 0,   1]])

###  3. Calculamos ahora la cinemática inversa


In [266]:
# La función nsolve de sympy busca valores busca soluciones que
# igualan una expresión a cero. Si requerimos T = D, entonces buscamos T-D = 0

T-D


Matrix([
[cos(q1 + q2 + q4) - sqrt(2)/2, -sin(q1 + q2 + q4) + sqrt(2)/2, 0, 0.467*cos(q1) + 0.4005*cos(q1 + q2) - 0.5],
[sin(q1 + q2 + q4) - sqrt(2)/2,  cos(q1 + q2 + q4) - sqrt(2)/2, 0, 0.467*sin(q1) + 0.4005*sin(q1 + q2) - 0.5],
[                            0,                              0, 0,                                0.134 - q3],
[                            0,                              0, 0,                                         0]])

In [267]:
# Buscamos una solución relajando la precisión requerida para facilitar el cálculo.
# La precisión por defecto es de 15 cifras significativas.

try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
q

Matrix([
[ 0.220089],
[  1.23996],
[    0.134],
[-0.674648]])

In [268]:
# enviamos los ángulos a las articulaciones
clientID = connect(19999)
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)


conectado a 19999


In [269]:
# verificamos la posición del actuador
retCode,pos=sim.simxGetObjectPosition(clientID, effector, -1, sim.simx_opmode_blocking)
print(pos)

[0.4999677836894989, 0.4999949336051941, 0.0997408926486969]


In [270]:
# y la orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, effector, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

0.0034228799750769087
0.0035820960897772315
45.00503231315234


Utilizzando la procedura per il calcolo della cinematica inversa, costruisci una sequenza di movimenti per allineare i tre cubi, uno sopra l'altro. Seguire la procedura riportata di seguito: 1. Prendi la posizione del primo cubo, prendilo da quella posizione e posizionalo in qualsiasi posizione tu decida. Ricordarsi di attivare e disattivare l'attuatore finale con la funzione setEffector() 2. Ottieni la posizione e l'orientamento del secondo cubo, prendilo dalla sua posizione e posizionalo sul primo cubo, assegnando l'orientamento necessario all'angolo gamma, in modo che sia perfettamente allineato con il primo cubo. 3. Ripetere la procedura per il terzo cubo, in modo che sia sul secondo e perfettamente allineato con gli altri due.

Nota: si consideri che l'unico orientamento che può essere gestito è l'angolo gamma (rotazione rispetto all'asse z, rispetto al sistema di coordinate del mondo). Suggerimento: ottieni automaticamente la posizione e l'orientamento di ogni cubo ed esegui calcoli da queste coordinate. Ricorda che le coordinate che otterrai saranno le coordinate del centro dell'oggetto.

In [271]:
# Para el primer cubo
clientID = connect(19999)

# obtenemos la posición
print("leyendo la posición objetivo...")
retCode,pos=sim.simxGetObjectPosition(clientID, caja1, -1, sim.simx_opmode_blocking)
print(pos)
# y orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, caja1, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

# definimos las coordenadas de destino
x = pos[0]
y = pos[1]
z = pos[2] + 0.026 # distancia del centro al borde + tolerancia
alpha = eul[0]
beta = eul[1]
gamma = eul[2]
t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)

# calculamos la cinemática inversa
print("calculando la cinemática inversa...")
try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
print(q)

# movemos el robot a la posición
print("moviendo a la posición objetivo...")
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# activamos el efector
setEffector(1)
time.sleep(1)
# levantamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# volvemos a la posición inicial
retCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el cubo
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# desactivamos el actuador
time.sleep(1)
setEffector(0)
# y levantamos
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
print("movimiento concluido 1!")

# Para el secondo cubo
#clientID = connect(19999)

# obtenemos la posición
print("leyendo la posición objetivo...")
retCode,pos=sim.simxGetObjectPosition(clientID, caja2, -1, sim.simx_opmode_blocking)
print(pos)
# y orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, caja2, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

# definimos las coordenadas de destino
x = pos[0]
y = pos[1]
z = pos[2] + 0.026 # distancia del centro al borde + tolerancia
alpha = eul[0]
beta = eul[1]
gamma = eul[2]
t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)

# calculamos la cinemática inversa
print("calculando la cinemática inversa...")
try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
print(q)

# movemos el robot a la posición
print("moviendo a la posición objetivo...")
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# activamos el efector
setEffector(1)
time.sleep(1)
# levantamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# volvemos a la posición inicial
retCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el cubo
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0.1, sim.simx_opmode_blocking)
time.sleep(1)
# desactivamos el actuador
time.sleep(1)
setEffector(0)
# y levantamos
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
print("movimiento concluido 2!")

# Para el terzo cubo
#clientID = connect(19999)

# obtenemos la posición
print("leyendo la posición objetivo...")
retCode,pos=sim.simxGetObjectPosition(clientID, caja3, -1, sim.simx_opmode_blocking)
print(pos)
# y orientación
retCode,eul=sim.simxGetObjectOrientation(clientID, caja3, -1, sim.simx_opmode_blocking)
print(eul[0]*180/3.1416)
print(eul[1]*180/3.1416)
print(eul[2]*180/3.1416)

# definimos las coordenadas de destino
x = pos[0]
y = pos[1]
z = pos[2] + 0.026 # distancia del centro al borde + tolerancia
alpha = eul[0]
beta = eul[1]
gamma = eul[2]
t = sp.Matrix([[1, 0, 0, x],
               [0, 1, 0, y], 
               [0, 0, 1, z], 
               [0, 0, 0, 1]])

D = t*matrixFromEuler(alpha, beta, gamma)

# calculamos la cinemática inversa
print("calculando la cinemática inversa...")
try:
    q = sp.nsolve((T-D), (q1, q2, q3, q4), (1, 1, 1, 1), prec=6)
except:
    print('no se encontró la solución')
    q = [0, 0, 0, 0]
print(q)

# movemos el robot a la posición
print("moviendo a la posición objetivo...")
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint4, q[3], sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_blocking)
time.sleep(1)
# activamos el efector
setEffector(1)
time.sleep(1)
# levantamos el actuador
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# volvemos a la posición inicial
retCode = sim.simxSetJointTargetPosition(clientID, joint1, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, 0, sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
time.sleep(1)
# bajamos el cubo
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0.05, sim.simx_opmode_blocking)
time.sleep(1)
# desactivamos el actuador
time.sleep(1)
setEffector(0)
# y levantamos
retCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_blocking)
print("movimiento concluido 3!")

conectado a 19999
leyendo la posición objetivo...
[0.5562732815742493, -0.058365389704704285, 0.02499992586672306]
-5.09177828497358e-07
-1.5504384127466685e-06
67.15154447475402
calculando la cinemática inversa...
Matrix([[-0.886861], [1.74719], [0.183000], [0.311686]])
moviendo a la posición objetivo...
movimiento concluido 1!
leyendo la posición objetivo...
[0.5624349117279053, 0.05748574808239937, 0.02499992400407791]
-9.823323209108158e-07
-1.1372287374596306e-06
-42.26803506395123
calculando la cinemática inversa...
Matrix([[-0.672940], [1.72878], [0.183000], [4.48963]])
moviendo a la posición objetivo...
movimiento concluido 2!
leyendo la posición objetivo...
[0.48865818977355957, -0.14840614795684814, 0.02499992400407791]
-1.0625309646642886e-06
-1.3413904568209847e-07
-100.2425793202621
calculando la cinemática inversa...
no se encontró la solución
[0, 0, 0, 0]
moviendo a la posición objetivo...
movimiento concluido 3!
